In [1]:
# Dictate the environment's loctaion of REFPROP
import os
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'

In [2]:
# Import the main class from the Python library
from ctREFPROP.ctREFPROP import REFPROPFunctionLibrary

# Imports from conda-installable packages
import pandas as pd

# Import numpy
import numpy as np

# Import matplotlib for plotting
import matplotlib.pyplot as plt

# Import Math for common values such as PI
import math


In [3]:
# Instantiate the library, and use the environment variable to explicitly state which path we want to use.
# As mentioned above, this will be changed to call the correct REFPROP functions to be used
# with MINI-REFPROP and 32-bit python.
# If using MINI-REFPROP and 32-bit python please make the following changes
# RP = REFPROPFunctionLibrary('C:/Program FIles (x86)/MINI-REFPROP\\REFPROP.DLL')
RP = REFPROPFunctionLibrary(os.environ['RPPREFIX'])

In [4]:
# This will call which root directory that will be used for the program. 
RP.SETPATHdll(os.environ['RPPREFIX'])

In [5]:
# Get the unit system we want to use (Mass base SI gives units in
# K, Pa, kg, m, N, J, W, and s)
MASS_BASE_SI = RP.GETENUMdll(0, "MASS BASE SI").iEnum

### Basic sCO2 States Derived

In [6]:
m_dot = .2 # kg/s of mass flow rate of sCO2
Q_in = 12 # kW of heat addition into the system

#### Outlet of Compressor, Inlet of Heat Source

In [7]:
# Temperature will be compared at end of script
T1 = 63 # [C]
P1 = 2500 # [psia]

T1 = T1 + 273.15 # Convert C to Kelvin
P1 = P1 * 6894.8 # convert psia to Pa

print("Pressure at Outlet of Compressor =", P1/6894.8, "psia")
print("Temperature at Outlet of Compressor =" , (T1 - 273.15) * (9/5) + 32, "F")

Pressure at Outlet of Compressor = 2500.0 psia
Temperature at Outlet of Compressor = 145.4 F


In [8]:
# Obtain fluid properties from the pressure and temperature outlined above
State_1 = RP.REFPROPdll("CO2","PT","D;V;H;S;CP/CV;W;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,P1,T1,[1.0])

# Outputs will be placed into data frame for organization
State_1 = pd.DataFrame(State_1.Output[0:9],
            index = ['Density [kg/m^3]', 'Volume [m^3/kg]', 'Enthalpy [J/kg]', 'Entropy [J/kg]',
                     'CP/CV', 'Speed of Sound', 'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['State 1'])

# Display the data frame
State_1

,State 1
Density [kg/m^3],646.530133
Volume [m^3/kg],0.001547
Enthalpy [J/kg],342482.845698
Entropy [J/kg],1412.054091
CP/CV,3.115189
Speed of Sound,346.935076
Thermal Cond. [W/(mK)],0.069987
Viscosity [Pa-s],0.000051
Prandtl,2.117334


#### Outlet of Heat Source, Inlet of Engine

In [9]:
P2 = P1
h2 = Q_in * 1000 / (m_dot) + State_1.loc['Enthalpy [J/kg]','State 1']

In [10]:
# Obtain fluid properties from the pressure and enthalpy outlined above
State_2 = RP.REFPROPdll("CO2","PH","T;D;V;S;CP/CV;W;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,P2,h2,[1.0])

# Outputs will be placed into data frame for organization
State_2 = pd.DataFrame(State_2.Output[0:9],
            index = ['Temperature [K]','Density [kg/m^3]', 'Volume [m^3/kg]', 'Entropy [J/kg]',
                     'CP/CV', 'Speed of Sound', 'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['State 2'])

# Display the data frame
State_2

,State 2
Temperature [K],356.611761
Density [kg/m^3],490.567738
Volume [m^3/kg],0.002038
Entropy [J/kg],1585.369760
CP/CV,3.008056
Speed of Sound,295.743887
Thermal Cond. [W/(mK)],0.056005
Viscosity [Pa-s],0.000037
Prandtl,1.817426


#### Outlet of Engine, Inlet of Heat Exchanger

In [11]:
# Engine Parameters
mass_cylinder = State_2.loc['Density [kg/m^3]', 'State 2'] * .000308276
State_3_den = mass_cylinder / .000454574

# With Isentropic expansion
State_3_entr = State_2.loc['Entropy [J/kg]', 'State 2']

State_3_isen = RP.REFPROPdll("CO2","DS","P;T;V;H;CP/CV;W;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,State_3_den,State_3_entr,[1.0])

# Outputs will be placed into data frame for organization
State_3_isen = pd.DataFrame(State_3_isen.Output[0:9],
            index = ['Pressure [Pa]', 'Temperature [K]', 'Volume [m^3/kg]', 'Enthalpy [J/kg]',
                     'CP/CV', 'Speed of Sound', 'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['State 3 Isentropic'])


In [12]:
# Account for Issentropic Efficiency
Isen_eff = .7
State_3_enth_new = h2 - (Isen_eff * (h2 - State_3_isen.loc['Enthalpy [J/kg]', 'State 3 Isentropic']))

# Using Pressure and enthalpy find the states of the fluid at the Outlet of Engine

State_3 = RP.REFPROPdll("CO2","PH","D;T;V;S;CP/CV;W;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,State_3_isen.loc['Pressure [Pa]', 'State 3 Isentropic'],State_3_enth_new,[1.0])

# Outputs will be placed into data frame for organization
State_3 = pd.DataFrame(State_3.Output[0:9],
            index = ['Density [kg/m^3]','Temperature [K]', 'Volume [m^3/kg]', 'Entropy [J/kg]',
                     'CP/CV', 'Speed of Sound', 'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['State 3'])


In [13]:
State_1.loc['Pressure [Pa]', 'State 1'] = P1
State_1.loc['Temperature [K]', 'State 1'] = T1
State_2.loc['Pressure [Pa]', 'State 2'] = P2
State_2.loc['Enthalpy [J/kg]', 'State 2'] = h2
State_3_isen.loc['Density [kg/m^3]', 'State 3 Isentropic'] = State_3_den
State_3_isen.loc['Entropy [J/kg]', 'State 3 Isentropic'] = State_3_entr
